In [1]:
import numpy as np
import os
import csv

In [2]:
vdd = 1
gnd = 0 

In [7]:
def readCSV(name):
    with open(name, 'r') as f:
        reader = csv.reader(f)
        data = list(reader)
    return data

In [17]:
single_nmos_root = '../MOSFET-sweeps/results-nmos/'
single_pmos_root = '../MOSFET-sweeps/results-pmos/'

Id = 6
Ib = 9
Ig = 7

def I_sub_pmos(width, gate, source, drain, voltage):
    data = readCSV(single_pmos_root + 'g'+str(gate)+ 's' + str(source) 
                   + 'd' + str(drain) + '-pmos_width_' + str(width)+ '.csv')
    row = voltage//0.01 + 1
    return data[row][Id]

def I_sub_nmos(width, gate, source, drain, voltage):
    data = readCSV(single_nmos_root + 'g'+str(gate)+ 's' + str(source) 
                   + 'd' + str(drain) + '-nmos_width_' + str(width)+ '.csv')
    row = voltage//0.01 + 1
    return data[row][Id]

def I_body_pmos(width, gate, source, drain, voltage):
    data = readCSV(single_pmos_root + 'g'+str(gate)+ 's' + str(source) 
                   + 'd' + str(drain) + '-pmos_width_' + str(width)+ '.csv')
    row = voltage//0.01 + 1
    return data[row][Ib]

def I_body_nmos(width, gate, source, drain, voltage):
    data = readCSV(single_nmos_root + 'g'+str(gate)+ 's' + str(source) 
                   + 'd' + str(drain) + '-nmos_width_' + str(width)+ '.csv')
    row = voltage//0.01 + 1
    return data[row][Ib]

def I_gate_pmos(width, gate, source, drain, voltage):
    data = readCSV(single_pmos_root + 'g'+str(gate)+ 's' + str(source) 
                   + 'd' + str(drain) + '-pmos_width_' + str(width)+ '.csv')
    row = voltage//0.01 + 1
    return data[row][Ig]

def I_gate_nmos(width, gate, source, drain, voltage):
    data = readCSV(single_nmos_root + 'g'+str(gate)+ 's' + str(source) 
                   + 'd' + str(drain) + '-nmos_width_' + str(width)+ '.csv')
    row = voltage//0.01 + 1
    return data[row][Ig]



In [23]:
stage_2_root = '../Stacked-MOS/'

int_voltage_nmos = csv.reader(open(stage_2_root+'NMOS/outputs.csv'))
int_voltage_pmos = csv.reader(open(stage_2_root+'PMOS/outputs.csv'))

int_voltage_nmos = list(int_voltage_nmos)
int_voltage_pmos = list(int_voltage_pmos)

vn = [float(row[-1]) for row in int_voltage_nmos[1:]]
vp = [float(row[-1]) for row in int_voltage_pmos[1:]]


In [24]:
def NOR(in1, in2):
    ### pmos width = 4, nmos width = 1
    if in1 == 0 and in2 == 0:
        sub_leakage = abs(I_sub_pmos(4, 0, 1, 's', vp[0]))
        body_leakage = 2*(I_body_nmos(1, 0, 0, 's', 1.0)) + I_body_pmos(4, 0, 1, 's', vp[0]) + I_body_pmos(4, 0, 's', 1, vp[0])
        gate_leakage = 2*(I_gate_nmos(1, 0, 0, 's', 1.0)) + I_gate_pmos(4, 0, 1, 's', vp[0]) + I_gate_pmos(4, 0, 's', 1, vp[0])
    
    if in1 == 0 and in2 == 1:
        sub_leakage = abs(I_sub_pmos(4, 0, 1, 's', vp[1]))
        body_leakage = I_body_nmos(1, 0, 0, 's', 0) + I_body_nmos(1, 1, 0, 's', 0) + I_body_pmos(4, 0, 1, 's', vp[1]) + I_body_pmos(4, 1, 's', 0, vp[1])
        gate_leakage = I_gate_nmos(1, 0, 0, 's', 0) + I_gate_nmos(1, 1, 0, 's', 0) + I_gate_pmos(4, 0, 1, 's', vp[1]) + I_gate_pmos(4, 1, 's', 0, vp[1])

    if in1 == 1 and in2 == 0:
        sub_leakage = abs(I_sub_pmos(4, 0, 1, 's', vp[2]))
        body_leakage = I_body_nmos(1, 0, 0, 's', 0) + I_body_nmos(1, 1, 0, 's', 0) + I_body_pmos(4, 0, 's', 0, vp[2]) + I_body_pmos(4, 1, 1, 's', vp[2])
        gate_leakage = I_gate_nmos(1, 0, 0, 's', 0) + I_gate_nmos(1, 1, 0, 's', 0) + I_gate_pmos(4, 0, 's', 0, vp[2]) + I_gate_pmos(4, 1, 1, 's', vp[2])

    if in1 == 1 and in2 == 1:
        sub_leakage = abs(I_sub_pmos(4, 1, 1, 's', vp[3]))
        body_leakage = 2*(I_body_nmos(1, 1, 0, 's', 0)) + I_body_pmos(4, 1, 1, 's', vp[3]) + I_body_pmos(4, 1, 's', 0, vp[3])
        gate_leakage = 2*(I_gate_nmos(1, 0, 0, 's', 0)) + I_gate_pmos(4, 1, 1, 's', vp[3]) + I_gate_pmos(4, 1, 's', 0, vp[3])
    
    return sub_leakage, body_leakage, gate_leakage

# NAND gate
def NAND(in1, in2):
    ### pmos width = 4, nmos width = 2
    if in1 == 0 and in2 == 0:
        sub_leakage = abs(I_sub_nmos(2, 0, 's', 1, vn[0]))
        body_leakage = 2*(I_body_pmos(4, 0, 1, 's', 1.0)) + I_body_nmos(2, 0, 0, 's', vn[0]) + I_body_nmos(2, 0, 's', 1, vn[0])
        gate_leakage = 2*(I_gate_pmos(4, 0, 1, 's', 1.0)) + I_gate_nmos(2, 0, 0, 's', vn[0]) + I_gate_nmos(2, 0, 's', 1, vn[0])
    
    if in1 == 0 and in2 == 1:
        sub_leakage = abs(I_sub_nmos(2, 0, 's', 1, vn[1]))
        body_leakage = I_body_nmos(2, 0, 's', 1, vn[1]) + I_body_nmos(2, 1, 0, 's', vn[1]) + I_body_pmos(4, 0, 1, 's', 1) + I_body_pmos(4, 1, 1, 's', 1)
        gate_leakage = I_gate_nmos(2, 0, 's', 1, vn[1]) + I_gate_nmos(2, 1, 0, 's', vn[1]) + I_gate_pmos(4, 0, 1, 's', 1) + I_gate_pmos(4, 1, 1, 's', 1)

    if in1 == 1 and in2 == 0:
        sub_leakage = abs(I_sub_nmos(2, 0, 0, 's', vn[2]))
        body_leakage = I_body_nmos(2, 0, 0, 's', vn[2]) + I_body_nmos(2, 1, 's', 1, vn[2]) + I_body_pmos(4, 0, 1, 's', 1) + I_body_pmos(4, 1, 1, 's', 1)
        gate_leakage = I_gate_nmos(2, 0, 0, 's', vn[2]) + I_gate_nmos(2, 1, 's', 1, vn[2]) + I_gate_pmos(4, 0, 1, 's', 1) + I_gate_pmos(4, 1, 1, 's', 1)

    if in1 == 1 and in2 == 1:
        sub_leakage = abs(I_sub_nmos(2, 1, 's', 0, vn[3]))
        body_leakage = 2*(I_body_pmos(4, 1, 1, 's', 0)) + I_body_nmos(2, 1, 's', 0, vn[3]) + I_body_nmos(2, 1, 0, 's', vn[3])
        gate_leakage = 2*(I_gate_pmos(4, 1, 1, 's', 0)) + I_gate_nmos(2, 1, 's', 0, vn[3]) + I_gate_nmos(2, 1, 0, 's', vn[3])
    
    return sub_leakage, body_leakage, gate_leakage

# NOT gate
def NOT(inp):
    ### pmos width = 2, nmos width = 1
    if inp == 0:
        sub_leakage = abs(I_sub_pmos(2, 0, 1, 's', 1))
        body_leakage = I_body_nmos(1, 0, 0, 's', 1.0) + I_body_pmos(2, 0, 1, 's', 1)
        gate_leakage = I_gate_nmos(1, 0, 0, 's', 1.0) + I_gate_pmos(2, 0, 1, 's', 1)
    
    if inp == 1:
        sub_leakage = abs(I_sub_pmos(2, 1, 1, 's', 0))
        body_leakage = I_body_nmos(1, 1, 0, 's', 0) + I_body_pmos(2, 1, 1, 's', 0)
        gate_leakage = I_gate_nmos(1, 1, 0, 's', 0) + I_gate_pmos(2, 1, 1, 's', 0)
    
    return sub_leakage, body_leakage, gate_leakage


In [21]:
def NAND_lite():
    pass 

def NOR_lite():
    pass

def NOT_lite():
    pass

In [22]:
def CLA_leakage(P0, P1, P2, P3, G0, G1, G2, G3, C0, Cin):
    I_leak = 0
    # add all the nand nor and not gates 
    
    